In [1]:
!pip install sentence-transformers datasets
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 857 kB/s 
     |████████████████████████████████| 451 kB 51.4 MB/s 
     |████████████████████████████████| 5.8 MB 58.0 MB/s 
     |████████████████████████████████| 1.3 MB 25.7 MB/s 
     |████████████████████████████████| 182 kB 49.4 MB/s 
     |████████████████████████████████| 7.6 MB 56.3 MB/s 
     |████████████████████████████████| 212 kB 53.1 MB/s 
     |████████████████████████████████| 132 kB 60.8 MB/s 
     |████████████████████████████████| 127 kB 49.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3015cc9ef206264819cdbdc682fb7fc19cb5ea4f0b66f884bfa287710e0f22b9
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
  Attempting uninstall: urllib3
    Found existing i

In [33]:
!pip install --upgrade pip
!pip install SpeechRecognition
!pip install pyautogui
!pip install clipboard
!pip install keyboard
!pip install pyaudio

  Using cached pip-22.3.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
D:\anaconda\envs\mldl\python.exe -m pip install --upgrade pip


  Using cached SpeechRecognition-3.9.0-py2.py3-none-any.whl (32.8 MB)
  Using cached PyAutoGUI-0.9.53-py3-none-any.whl
  Using cached PyScreeze-0.1.28-py3-none-any.whl
  Using cached pytweening-1.0.4-py3-none-any.whl
  Using cached MouseInfo-0.1.3-py3-none-any.whl
  Using cached PyGetWindow-0.0.9-py3-none-any.whl
  Using cached PyMsgBox-1.0.9-py3-none-any.whl
  Using cached PyRect-0.2.0-py2.py3-none-any.whl
  Using cached pyperclip-1.8.2-py3-none-any.whl
  Using cached clipboard-0.0.4-py3-none-any.whl
  Using cached keyboard-0.13.5-py3-none-any.whl (58 kB)
  Using cached PyAudio-0.2.12-cp38-cp38-win_amd64.whl (163 kB)


In [20]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sklearn.cluster import KMeans

from speech_recognition import *
from pyautogui import *
import clipboard
import keyboard
import pyaudio
import time # 필요한 모듈 불러오기
from datetime import datetime

# AttributeError: module 'datetime' has no attribute 'now' 이 오류 나오면
# from datetime import datetime


In [21]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [22]:
# 모델 설정
# 활용할 모델을 선택
# 본 노트북에서는 klue-roberta-base 모델을 활용하지만, 
# https://huggingface.co/klue
# 페이지에서 더 다양한 사전학습 언어 모델을 확인하실 수 있습니다.

'''
klue/roberta-base
klue/roberta-large
klue/roberta-small
klue/bert-base
'''

model_name = "klue/roberta-base"

train_batch_size = 32
num_epochs = 4
model_save_path = "output/training_klue_sts_" + model_name.replace("/", "-") + "-" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

embedding_model = models.Transformer(model_name)

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for

In [23]:
pooler = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)


In [24]:
model = SentenceTransformer(modules=[embedding_model, pooler])
datasets = load_dataset("klue", "sts")

2022-12-09 00:15:22 - Use pytorch device: cuda
2022-12-09 00:15:26 - Found cached dataset klue (C:/Users/maili/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
testsets = load_dataset("kor_nlu", "sts")

2022-12-09 00:15:30 - Found cached dataset kor_nlu (C:/Users/maili/.cache/huggingface/datasets/kor_nlu/sts/1.0.0/4facbba77df60b0658056ced2052633e681a50187b9428bd5752ebd59d332ba8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
train_samples = []
dev_samples = []
test_samples = []

# KLUE STS 내 훈련, 검증 데이터 예제 변환
for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화

        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]], 
            label=score,
        )

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

# KorSTS 내 테스트 데이터 예제 변환
for example in testsets["test"]:
    score = float(example["score"]) / 5.0

    if example["sentence1"] and example["sentence2"]:
        inp_example = InputExample(
            texts=[example["sentence1"], example["sentence2"]],
            label=score,
        )

    test_samples.append(inp_example)

In [27]:
train_dataloader = DataLoader(
    train_samples,
    shuffle=True,
    batch_size=train_batch_size,
)
train_loss = losses.CosineSimilarityLoss(model=model)


In [28]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_samples,
    name="sts-dev",
)

In [29]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)  # 10% of train data for warm-up
logging.info(f"Warmup-steps: {warmup_steps}")

2022-12-09 00:15:31 - Warmup-steps: 37


In [30]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=model_save_path,
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/365 [00:00<?, ?it/s]

2022-12-09 00:17:00 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-12-09 00:17:01 - Cosine-Similarity :	Pearson: 0.8645	Spearman: 0.8606
2022-12-09 00:17:01 - Manhattan-Distance:	Pearson: 0.8655	Spearman: 0.8597
2022-12-09 00:17:01 - Euclidean-Distance:	Pearson: 0.8661	Spearman: 0.8599
2022-12-09 00:17:01 - Dot-Product-Similarity:	Pearson: 0.8556	Spearman: 0.8479
2022-12-09 00:17:01 - Save model to output/training_klue_sts_klue-roberta-base-2022-12-09_00-15-19


In [32]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

2022-12-09 00:17:06 - Load pretrained SentenceTransformer: output/training_klue_sts_klue-roberta-base-2022-12-09_00-15-19
2022-12-09 00:17:06 - Use pytorch device: cuda
2022-12-09 00:17:06 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-12-09 00:17:09 - Cosine-Similarity :	Pearson: 0.7588	Spearman: 0.7483
2022-12-09 00:17:09 - Manhattan-Distance:	Pearson: 0.7573	Spearman: 0.7538
2022-12-09 00:17:09 - Euclidean-Distance:	Pearson: 0.7569	Spearman: 0.7531
2022-12-09 00:17:09 - Dot-Product-Similarity:	Pearson: 0.7348	Spearman: 0.7226


0.7538184018566648

In [ ]:
docs = []
dic = {}

dir_path = "./data/"

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        dic[file_path] = 0

for i in dic:
    file= open(i, 'rt' , encoding='UTF8')
    passwords = file.readlines()
    for password in passwords:
    #print(password)
        password = password.replace('\n', '')
        docs.append(password)

print(docs)
# document_embeddings = model.encode(docs)





In [4]:
docs = []

dir_path = "./data/"

for (root, directories, files) in os.walk(dir_path):
    for file in files:
        file_path = os.path.join(root, file)
        print(file_path)

./data/내과\심근경색\심근경색.txt
./data/내과\심정지\심정지.txt
./data/내과\압좌증후군\압좌증후군.txt
./data/내과\압좌증후군\압좌증후군2.txt
./data/외과\골절\골절.txt


In [34]:
def State(voice):
    query = voice
    query_embedding = model.encode(query)
    top_k = min(len(docs), len(docs))
    # 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
    cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

    # 코사인 유사도 순으로 `top_k` 개 문장 추출
    top_results = torch.topk(cos_scores, k=top_k)
    
    print(f"입력 문장: {query}")
    print(f"\n<입력 문장과 비교한 {top_k} 개의 문장>\n")
    
    for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
        if score > 0.5:
            print(f"{i+1}: {docs[idx]} \n{'(유사도: {:.4f})'.format(score)}\n")
    
def read_voice(): # 음성 인식을 하는 함수
    r = Recognizer()
    mic = Microphone() # 마이크 객체
    with mic as source:
        audio = r.listen(source) # 음성 읽어오기
    voice_data = r.recognize_google(audio, language='ko')
    return voice_data # 값 반환



voice = read_voice() # 음성 인식
State(voice)
# 버스안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 가래가 나와요 



result2:
{   'alternative': [   {   'confidence': 0.92365944,
                           'transcript': '버스 안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 가래가 '
                                         '나와요'},
                       {   'transcript': '버스 안에 사람이 발작을 일으키고 쓰러졌어요 집에서 계속 가래가 '
                                         '나와요'},
                       {   'transcript': '버스 안에 사람이 발작을 일으켜서 쓰러졌어요 입에서 계속 가래가 '
                                         '나와요'},
                       {   'transcript': '버스 안에 사람이 발적을 일으키고 쓰러졌어요 입에서 계속 가래가 '
                                         '나와요'},
                       {   'transcript': '버스 안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 아래가 '
                                         '나와요'}],
    'final': True}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

입력 문장: 버스 안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 가래가 나와요

<입력 문장과 비교한 8 개의 문장>

1: 지나가던 여자분이 발작을 일으키더니 입에서 가래가 나오면서 쓰러졌습니다. 
(유사도: 0.7346)

2: 갑자기 친구가 발작을 일으키더니 쓰려졌습니다. 입에서 가래가 나오고 있습니다. () 
(유사도: 0.6890)

3: 길을 가는데 남자분 한분이 쓰러져 있습니다. 입에 가래가 나오고 있습니다. 
(유사도: 0.6049)

4: 도서관에서 공부하는데 친구가 갑자기 쓰러지더니 숨을 쉬지 않습니다. 심폐소생술을 하고 있는데 입에서 가래가 나옵니다 
(유사도: 0.5286)



In [39]:
query = "버스안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 가래가 나와요 "
query_embedding = model.encode(query)


top_k = min(len(docs), len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 비교한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    if score > 0.5:
        print(f"{i+1}: {docs[idx]} \n{'(유사도: {:.4f})'.format(score)}\n")



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

입력 문장: 버스안에 사람이 발작을 일으키고 쓰러졌어요 입에서 계속 가래가 나와요 

<입력 문장과 비교한 8 개의 문장>

1: 지나가던 여자분이 발작을 일으키더니 입에서 가래가 나오면서 쓰러졌습니다. 
(유사도: 0.7276)

2: 갑자기 친구가 발작을 일으키더니 쓰려졌습니다. 입에서 가래가 나오고 있습니다. 
(유사도: 0.7003)

3: 길을 가는데 남자분 한분이 쓰러져 있습니다. 입에 가래가 나오고 있습니다. 
(유사도: 0.5894)

4: 도서관에서 공부하는데 친구가 갑자기 쓰러지더니 숨을 쉬지 않습니다. 심폐소생술을 하고 있는데 입에서 가래가 나옵니다 
(유사도: 0.5193)



In [26]:
document_embeddings = model.encode(docs)

num_clusters = 3

k_means = KMeans(n_clusters=num_clusters)
k_means.fit(document_embeddings)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

D:\anaconda\envs\mldl\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=3)

In [27]:
cluster_assignment = k_means.labels_

In [28]:
cluster_assignment

array([1, 2, 1, 0, 1, 0, 0])

In [29]:
import torch
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Huffon/sentence-klue-roberta-base")

docs = [
    "1992년 7월 8일 손흥민은 강원도 춘천시 후평동에서 아버지 손웅정과 어머니 길은자의 차남으로 태어나 그곳에서 자랐다.",
    "형은 손흥윤이다.",
    "춘천 부안초등학교를 졸업했고, 춘천 후평중학교에 입학한 후 2학년때 원주 육민관중학교 축구부에 들어가기 위해 전학하여 졸업하였으며, 2008년 당시 FC 서울의 U-18팀이었던 동북고등학교 축구부에서 선수 활동 중 대한축구협회 우수선수 해외유학 프로젝트에 선발되어 2008년 8월 독일 분데스리가의 함부르크 유소년팀에 입단하였다.",
    "함부르크 유스팀 주전 공격수로 2008년 6월 네덜란드에서 열린 4개국 경기에서 4게임에 출전, 3골을 터뜨렸다.",
    "1년간의 유학 후 2009년 8월 한국으로 돌아온 후 10월에 개막한 FIFA U-17 월드컵에 출전하여 3골을 터트리며 한국을 8강으로 이끌었다.",
    "그해 11월 함부르크의 정식 유소년팀 선수 계약을 체결하였으며 독일 U-19 리그 4경기 2골을 넣고 2군 리그에 출전을 시작했다.",
    "독일 U-19 리그에서 손흥민은 11경기 6골, 2부 리그에서는 6경기 1골을 넣으며 재능을 인정받아 2010년 6월 17세의 나이로 함부르크의 1군 팀 훈련에 참가, 프리시즌 활약으로 함부르크와 정식 계약을 한 후 10월 18세에 함부르크 1군 소속으로 독일 분데스리가에 데뷔하였다.",
]
document_embeddings = model.encode(docs)

query = "손흥민은 어린 나이에 유럽에 진출하였다."
query_embedding = model.encode(query)

top_k = min(5, len(docs))

# 입력 문장 - 문장 후보군 간 코사인 유사도 계산 후,
cos_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]

# 코사인 유사도 순으로 `top_k` 개 문장 추출
top_results = torch.topk(cos_scores, k=top_k)

print(f"입력 문장: {query}")
print(f"\n<입력 문장과 유사한 {top_k} 개의 문장>\n")

for i, (score, idx) in enumerate(zip(top_results[0], top_results[1])):
    print(f"{i+1}: {docs[idx]} {'(유사도: {:.4f})'.format(score)}\n")

2022-12-08 23:37:42 - Load pretrained SentenceTransformer: Huffon/sentence-klue-roberta-base


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

ConnectionError: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.